In [30]:
import joblib
import pandas as pd

from rdkit import Chem
from sklearn.metrics import roc_auc_score
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, global_add_pool
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F

import numpy as np
from rdkit.Chem import rdFingerprintGenerator
import deepchem as dc
import random
import optuna

from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    log_loss,
    roc_auc_score,
    precision_score,
    recall_score,
    confusion_matrix,
)

from pprint import pprint

In [2]:
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


# Load pre-trained models

In [4]:
# load the results back
loaded_results = joblib.load("models/tpot_results_mixed.joblib")

# Convert results to DataFrame
tpot_df = pd.DataFrame(loaded_results)
tpot_df

,Best model,PCA Components,Model Name,Parameters,AUC,Precision,Recall,Sensitivity,Specificity
0,"(MaxAbsScaler(), KNeighborsClassifier(n_neighb...",10,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.782497,0.747525,0.834254,0.834254,0.592
1,"(StandardScaler(), ([DecisionTreeRegressor(cri...",20,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.792309,0.762887,0.817680,0.817680,0.632
2,"(PolynomialFeatures(include_bias=False), (Extr...",50,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.807691,0.750000,0.878453,0.878453,0.576
3,"((ExtraTreeClassifier(criterion='entropy', max...",100,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.792000,0.766169,0.850829,0.850829,0.624
4,"((ExtraTreeClassifier(criterion='entropy', max...",200,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.793812,0.740385,0.850829,0.850829,0.568
5,"(KNeighborsClassifier(n_neighbors=3, p=1, weig...",500,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.763315,0.791908,0.756906,0.756906,0.712
6,"(KNeighborsClassifier(n_neighbors=23, weights=...",1000,KNeighborsClassifier,"{'algorithm': 'auto', 'leaf_size': 30, 'metric...",0.783006,0.750000,0.812155,0.812155,0.608


In [5]:
model1 = loaded_results[2]["Best model"]
model1

Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(include_bias=False)),
                ('extratreesclassifier',
                 ExtraTreesClassifier(max_features=0.9500000000000001,
                                      min_samples_leaf=10, min_samples_split=11,
                                      random_state=42))])

In [8]:
class GCN(torch.nn.Module):
    def __init__(
        self,
        num_node_features,
        num_classes,
        num_layers=3,
        hidden_dim=64,
        dropout_prob=0.5,
        activation="relu",
    ):
        super(GCN, self).__init__()

        # Store activation function dynamically
        if activation == "relu":
            self.activation = F.relu
        elif activation == "tanh":
            self.activation = F.tanh
        else:
            raise ValueError("Unsupported activation function")

        self.dropout_prob = dropout_prob

        # Dynamically define the GCN layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(num_node_features, hidden_dim))
        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))

        # Final fully connected layer
        self.fc = torch.nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # Apply GCN layers dynamically
        for conv in self.convs:
            x = conv(x, edge_index)
            x = self.activation(x)

        # Global pooling (combine different pooling methods)
        x = torch.cat([global_mean_pool(x, batch), global_add_pool(x, batch)], dim=1)

        # Apply dropout
        x = F.dropout(x, p=self.dropout_prob, training=self.training)

        # Final classification layer
        return F.log_softmax(self.fc(x), dim=1)



In [7]:
# Suggest hyperparameters
trial = joblib.load("models/graph_gcn_mixed_filtered_outliers.joblib")
best_params = trial.params
best_params

{'hidden_dim': 231,
 'dropout_prob': 0.3543332425071246,
 'lr': 0.0007656402736244026,
 'weight_decay': 1.7118701796851756e-05,
 'num_layers': 4,
 'activation': 'relu'}

In [9]:
model2 = GCN(
    num_node_features=70,
    num_classes=2,
    num_layers=best_params["num_layers"],
    hidden_dim=best_params["hidden_dim"],
    dropout_prob=best_params["dropout_prob"],
    activation=best_params["activation"],
).to(device)

model2.load_state_dict(torch.load("models/graph_gcn_mixed_filtered_outliers.pth"))

/tmp/ipykernel_83057/4229443039.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load("models/graph_gcn_mixed_filtered_outliers.pth"))


<All keys matched successfully>

# Data preparation

## PCA prep

In [10]:
# Load pd_train
pd_train = pd.read_parquet("data/training_class_mixed.parquet")
pd_train["label"] = pd_train["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0)
print(pd_train.shape)

(1221, 16095)


In [11]:
# Load pd_test
pd_test = pd.read_parquet("data/testing_class_mixed.parquet")
pd_test["label"] = pd_test["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0)
print(pd_test.shape)

(306, 16095)


In [13]:
X_train = pd_train.drop(columns=["Liver", "label", "Smiles"])
y_train = pd_train["label"]

X_test = pd_test.drop(columns=["Liver", "label", "Smiles"])
y_test = pd_test["label"]

In [14]:
# get X PCA components 50
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

## Graph prep

In [15]:
# Featurization using DeepChem's MolGraphConvFeaturizer
from utils.SmilesEnumeration import SmilesEnumerator


def featurize_smiles(smiles):
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
    graph_data = featurizer.featurize([smiles])[0]

    # Get DeepChem atom features
    atom_features_deepchem = graph_data.node_features

    return atom_features_deepchem


# Function to generate Morgan Fingerprints (ECFP)
def generate_ecfp(smiles):
    # Morgan fingerprint generator
    mfgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=4096)

    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return None
    return mfgen.GetFingerprintAsNumPy(molecule)


# Function to convert SMILES to PyTorch Geometric Data object using DeepChem featurizer
def smiles_to_graph_featurizer(smiles):
    # Featurization using DeepChem
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)

    # Featurize the SMILES string using DeepChem
    graph_data = featurizer.featurize([smiles])[0]
    return graph_data.node_features, graph_data.edge_features, graph_data.edge_index


# Function to extract atom features
def atom_features(atom, ecfp):
    # Get the atom index for corresponding ECFP value
    atom_idx = atom.GetIdx()

    return [
        atom.GetAtomicNum(),  # Atomic number
        atom.GetDegree(),  # Number of bonds
        atom.GetTotalNumHs(),  # Total number of hydrogens
        atom.GetFormalCharge(),  # Formal charge of the atom
        atom.GetImplicitValence(),  # Implicit valence
        atom.GetNumRadicalElectrons(),  # Number of radical electrons
        int(atom.GetIsAromatic()),  # Is the atom aromatic?
        atom.GetMass(),  # Atomic mass
        atom.GetHybridization().real,  # Hybridization state (SP, SP2, SP3, etc.)
        ecfp[atom_idx],  # Morgan fingerprint (ECFP) for the atom
    ]


# Function to extract bond features
def bond_features(bond):
    bond_type = bond.GetBondTypeAsDouble()  # Bond type as a float
    is_aromatic = bond.GetIsAromatic()  # Aromatic bond
    is_conjugated = bond.GetIsConjugated()  # Conjugated bond
    is_in_ring = bond.IsInRing()  # Whether the bond is part of a ring
    stereo = bond.GetStereo()  # Bond stereochemistry

    # Convert stereo information to a one-hot encoded format
    stereo_one_hot = [0, 0, 0, 0]  # Stereo options: None, E, Z, Other
    if stereo == Chem.BondStereo.STEREONONE:
        stereo_one_hot[0] = 1
    elif stereo == Chem.BondStereo.STEREOE:
        stereo_one_hot[1] = 1
    elif stereo == Chem.BondStereo.STEREOZ:
        stereo_one_hot[2] = 1
    else:
        stereo_one_hot[3] = 1

    # Combine all features into a single tensor
    return [
        bond_type,
        float(is_aromatic),
        float(is_conjugated),
        float(is_in_ring),
    ] + stereo_one_hot


# Convert SMILES to PyTorch Geometric Data object
def smiles_to_graph(smiles, label):
    mol = Chem.MolFromSmiles(smiles)

    atom_features_list = []
    edge_index = []
    edge_attr = []

    # DeepChem features
    atom_features_deepchem = featurize_smiles(smiles)

    # Generate Morgan Fingerprint (ECFP)
    ecfp_features = generate_ecfp(smiles)

    # Generate Molecule Graph Convolution features
    mol_graph_node_features, mol_graph_edge_features, mol_graph_edge_index = (
        smiles_to_graph_featurizer(smiles)
    )

    # Nodes (atoms)
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_features(atom, ecfp_features))

    atom_features_list = np.array(atom_features_list)

    # Edges (bonds)
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()

        # Append bidirectional edges for undirected graphs
        edge_index.append([i, j])
        edge_index.append([j, i])

        # Append bond features for both directions
        edge_attr.append(bond_features(bond))
        edge_attr.append(bond_features(bond))

    # Convert atom features to a tensor
    combined_features = np.concatenate(
        (atom_features_list, atom_features_deepchem, mol_graph_node_features), axis=1
    )
    x = torch.tensor(combined_features, dtype=torch.float)

    # Convert edge indices and features to tensors, handle empty edge case
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    # combine edge features from ECFP and MolGraphConv
    edge_attr = np.array(edge_attr)
    edge_attr = np.concatenate((edge_attr, mol_graph_edge_features), axis=1)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)

    # Label (target)
    y = torch.tensor([label], dtype=torch.long)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)



# Function to load data from parquet and apply SMILES augmentation for training
def load_data_from_parquet(file_path):
    df = pd.read_parquet(file_path)

    smiles_list = df["Smiles"].values
    labels = df["Liver"].apply(lambda x: 1 if x == "Hepatotoxicity" else 0).values

    data_list = []

    # Initialize the SmilesEnumerator for data augmentation
    for smiles, label in zip(smiles_list, labels):
        # For test data, no augmentation, just use canonical SMILES
        graph_data = smiles_to_graph(smiles, label)
        data_list.append(graph_data)

    return data_list

In [16]:
# Load training and testing data
training_data = load_data_from_parquet("data/training_class_mixed.parquet")
testing_data = load_data_from_parquet("data/testing_class_mixed.parquet")

In [17]:
# Create data loaders
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=32, shuffle=False)

# Model training

In [18]:
tpot_preds = model1.predict_proba(X_train_pca)[:, 1]

# Predict on validation set
model2.eval()
gcn_preds = []
with torch.no_grad():
    for data in train_loader:
        data = data.to(device)
        out = model2(data)
        gcn_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

gcn_preds = np.array(gcn_preds)

# Combine predictions as stacked features
stacked_features_train = np.vstack([tpot_preds, gcn_preds]).T
stacked_features_train.shape

(1221, 2)

In [19]:
# Train meta-model
meta_model = LogisticRegression(random_state=42)
meta_model.fit(stacked_features_train, y_train)

LogisticRegression(random_state=42)

In [20]:
from sklearn.metrics import roc_auc_score, accuracy_score

# TPOT predictions on test set
tpot_test_preds = model1.predict_proba(X_test_pca)[:, 1]

# GCN predictions on test set
gcn_test_preds = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model2(data)
        gcn_test_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

# Stack test predictions
stacked_features_test = np.vstack(
    [tpot_test_preds, gcn_test_preds]
).T

# Make final predictions with meta-model
final_predictions = meta_model.predict(stacked_features_test)
final_auc = roc_auc_score(y_test, final_predictions)
final_accuracy = accuracy_score(y_test, final_predictions)

print("Stacked Model Test AUC:", final_auc)
print("Stacked Model Test Accuracy:", final_accuracy)

Stacked Model Test AUC: 0.7481767955801106
Stacked Model Test Accuracy: 0.7679738562091504


# Optuna

In [21]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def compute_loss(y_hat, y):
    y_hat = np.clip(y_hat, 1e-7, 1 - 1e-7)

    return (-y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)).mean()


def predict(X, theta):
    dot_product = np.dot(X, theta)
    y_hat = sigmoid(dot_product)

    return y_hat


def compute_gradient(X, y, y_hat):
    return np.dot(X.T, (y_hat - y)) / y.size


def update_theta(theta, gradient, lr):
    return theta - lr * gradient


def compute_accuracy(X, y, theta):
    y_hat = predict(X, theta).round()
    acc = (y_hat == y).mean()

    return acc

In [23]:
# Define inference function for each model
def get_model_predictions(
    tpot_model,
    gcn_model,
    X_pca,
    loader,
    device,
):
    # TPOT model predictions
    tpot_preds = tpot_model.predict_proba(X_pca)[:, 1]

    # GCN model predictions
    gcn_preds = []
    gcn_model.eval()
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = gcn_model(data)
            gcn_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1
    gcn_preds = np.array(gcn_preds)

    # Combine predictions into stacked feature set
    stacked_features = np.vstack([tpot_preds, gcn_preds]).T
    return stacked_features

In [25]:
# Optuna objective function to optimize the final stacking ensemble
def objective(trial):
    # Hyperparameter suggestions for stacking model
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    max_iter = trial.suggest_int("max_iter", 50, 200)

    # Prepare stacked training features and target
    stacked_train = get_model_predictions(
        model1,
        model2,
        X_train_pca,
        train_loader,
        device,
    )

    # Initialize stacking model (Logistic Regression)
    stacking_model = LogisticRegression(C=lr, max_iter=max_iter, random_state=42)

    # Fit stacking model on stacked features
    stacking_model.fit(stacked_train, y_train)

    # Get validation stacked features for AUC evaluation
    stacked_val = get_model_predictions(
        model1, model2, X_test_pca, test_loader, device
    )
    val_preds = stacking_model.predict_proba(stacked_val)[:, 1]
    val_auc = roc_auc_score(y_test, val_preds)

    # Report AUC for Optuna and check for pruning
    trial.report(val_auc, step=0)
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    return val_auc

In [26]:
# Run Optuna study to find the best parameters for the stacking model
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2024-11-16 16:04:24,406] A new study created in memory with name: no-name-688779a4-af10-4620-8b0f-7f84b2b64847
/tmp/ipykernel_83057/3851873801.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
[I 2024-11-16 16:04:24,537] Trial 0 finished with value: 0.8061878453038674 and parameters: {'lr': 0.0005629755350382936, 'max_iter': 197}. Best is trial 0 with value: 0.8061878453038674.
/tmp/ipykernel_83057/3851873801.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
[I 2024-11-16 16:04:24,654] Trial 1 finished with value: 0.8046850828729282 and parameters: {'lr': 0.00536923110

In [27]:
# Best trial results
print("Best trial:")
print(f"AUC Score: {study.best_trial.value}")
print("Best Parameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

Best trial:
AUC Score: 0.8082209944751382
Best Parameters:
  lr: 3.730389920476909e-05
  max_iter: 193


In [28]:
from sklearn.metrics import f1_score


def evaluate_with_optimal_threshold(y_true, y_pred_proba):
    # Calculate AUC
    auc = roc_auc_score(y_true, y_pred_proba)

    # Initialize variables to store the best threshold and performance metrics
    best_threshold = 0.5
    best_metrics = {
        "accuracy": 0,
        "precision": 0,
        "recall": 0,
        "sensitivity": 0,
        "specificity": 0,
        "f1": 0,
    }

    # Iterate over possible thresholds
    for threshold in np.arange(0.0, 1.0, 0.01):
        # Binarize predictions based on the current threshold
        y_pred = (y_pred_proba >= threshold).astype(int)

        # Calculate precision, recall, accuracy, sensitivity, and specificity
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)  # Same as sensitivity for positive class
        accuracy = accuracy_score(y_true, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = f1_score(y_true, y_pred)

        # Update the best threshold if F1 score is higher than previous best
        if f1 > best_metrics["f1"]:
            best_threshold = threshold
            best_metrics = {
                "accuracy": accuracy,
                "precision": precision,
                "recall": recall,
                "sensitivity": sensitivity,
                "specificity": specificity,
                "f1": f1,
            }

    # Compile all results
    result = {
        "Best Threshold": best_threshold,
        "AUC": auc,
        **best_metrics,
    }
    return result

In [31]:
from sklearn.metrics import roc_auc_score, accuracy_score

# TPOT predictions on test set
tpot_test_preds = model1.predict_proba(X_test_pca)[:, 1]

# GCN predictions on test set
gcn_test_preds = []
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model2(data)
        gcn_test_preds.extend(out[:, 1].cpu().numpy())  # Probability for class 1

# Stack test predictions
stacked_features_test = np.vstack(
    [tpot_test_preds, gcn_test_preds]
).T

# Make final predictions with meta-model
final_predictions = meta_model.predict(stacked_features_test)

# Evaluate metrics
result = evaluate_with_optimal_threshold(y_test, final_predictions)
pprint(result)

{'AUC': 0.7481767955801106,
 'Best Threshold': 0.01,
 'accuracy': 0.7679738562091504,
 'f1': 0.8136482939632546,
 'precision': 0.775,
 'recall': 0.856353591160221,
 'sensitivity': 0.856353591160221,
 'specificity': 0.64}
